# 第4章: 形態素解析

夏目漱石の小説『吾輩は猫である』の文章（neko.txt）をMeCabを使って形態素解析し，その結果をneko.txt.mecabというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

なお，問題37, 38, 39はmatplotlibもしくはGnuplotを用いるとよい．

## MeCab動作チェック

In [1]:
import MeCab
import sys
import re
from collections import Counter

In [2]:
text = 'pythonが大好きです。'

In [3]:
print('表層形\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音')
parse = MeCab.Tagger().parse(text)
print(parse)

表層形	品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音
python	名詞,固有名詞,組織,*,*,*,*
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
大好き	名詞,形容動詞語幹,*,*,*,*,大好き,ダイスキ,ダイスキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。
EOS



In [4]:
lines = parse.split('\n')
lines

['python\t名詞,固有名詞,組織,*,*,*,*',
 'が\t助詞,格助詞,一般,*,*,*,が,ガ,ガ',
 '大好き\t名詞,形容動詞語幹,*,*,*,*,大好き,ダイスキ,ダイスキ',
 'です\t助動詞,*,*,*,特殊・デス,基本形,です,デス,デス',
 '。\t記号,句点,*,*,*,*,。,。,。',
 'EOS',
 '']

## MeCab実行

コマンドプロンプトで以下を実行

`mecab neko.txt -o neko.txt.mecab`

## 30. 形態素解析結果の読み込み
形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．

In [80]:
import os
from typing import List, Dict

In [81]:
input_path = os.path.join(os.getcwd(), '../data/neko.txt.mecab')

In [82]:
# 読み込み
def get_raw(path: str) -> str:
    with open(path, mode='r', encoding='utf-8') as f:
        return f.read()

In [83]:
# 文ごとにリストに入れる
def get_sentences(text: str) -> List[str]:
    return [s for s in text.split('EOS\n') if s != '']

test_text = """
一	名詞,数,*,*,*,*,一,イチ,イチ
EOS
EOS
　	記号,空白,*,*,*,*,　,　,　
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。
EOS
名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
まだ	副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ
無い	形容詞,自立,*,*,形容詞・アウオ段,基本形,無い,ナイ,ナイ
。	記号,句点,*,*,*,*,。,。,。
EOS
EOS
"""
get_sentences(test_text)

['\n一\t名詞,数,*,*,*,*,一,イチ,イチ\n',
 '\u3000\t記号,空白,*,*,*,*,\u3000,\u3000,\u3000\n吾輩\t名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ\nは\t助詞,係助詞,*,*,*,*,は,ハ,ワ\n猫\t名詞,一般,*,*,*,*,猫,ネコ,ネコ\nで\t助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ\nある\t助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル\n。\t記号,句点,*,*,*,*,。,。,。\n',
 '名前\t名詞,一般,*,*,*,*,名前,ナマエ,ナマエ\nは\t助詞,係助詞,*,*,*,*,は,ハ,ワ\nまだ\t副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ\n無い\t形容詞,自立,*,*,形容詞・アウオ段,基本形,無い,ナイ,ナイ\n。\t記号,句点,*,*,*,*,。,。,。\n']

In [84]:
# 文の形態素をマッピングし、リストに入れる
def mapping_sentence(text: str) -> List[Dict]:
    morphemes = text.split('\n')
    ret_list = []
    for m in morphemes:
        # 空白行は除外
        if m == '':
            continue
        # 表層系とそれ以外を分離
        surface, others = m.split('\t')
        # それ以外の中身を分離
        other = others.split(',')
        # 空白は除外
        if other[0] == '記号' and other[1] == '空白':
            continue
        # マッピング型にしてリストに追加
        ret_list.append({
            'surface': surface,
            'base': other[6],
            'pos': other[0],
            'pos1': other[1],
        })
    return ret_list

test_text = """
　	記号,空白,*,*,*,*,　,　,　
どこ	名詞,代名詞,一般,*,*,*,どこ,ドコ,ドコ
で	助詞,格助詞,一般,*,*,*,で,デ,デ
生れ	動詞,自立,*,*,一段,連用形,生れる,ウマレ,ウマレ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
か	助詞,副助詞／並立助詞／終助詞,*,*,*,*,か,カ,カ
とんと	副詞,一般,*,*,*,*,とんと,トント,トント
見当	名詞,サ変接続,*,*,*,*,見当,ケントウ,ケントー
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
つか	動詞,自立,*,*,五段・カ行イ音便,未然形,つく,ツカ,ツカ
ぬ	助動詞,*,*,*,特殊・ヌ,基本形,ぬ,ヌ,ヌ
。	記号,句点,*,*,*,*,。,。,。
"""

mapping_sentence(test_text)

[{'surface': 'どこ', 'base': 'どこ', 'pos': '名詞', 'pos1': '代名詞'},
 {'surface': 'で', 'base': 'で', 'pos': '助詞', 'pos1': '格助詞'},
 {'surface': '生れ', 'base': '生れる', 'pos': '動詞', 'pos1': '自立'},
 {'surface': 'た', 'base': 'た', 'pos': '助動詞', 'pos1': '*'},
 {'surface': 'か', 'base': 'か', 'pos': '助詞', 'pos1': '副助詞／並立助詞／終助詞'},
 {'surface': 'とんと', 'base': 'とんと', 'pos': '副詞', 'pos1': '一般'},
 {'surface': '見当', 'base': '見当', 'pos': '名詞', 'pos1': 'サ変接続'},
 {'surface': 'が', 'base': 'が', 'pos': '助詞', 'pos1': '格助詞'},
 {'surface': 'つか', 'base': 'つく', 'pos': '動詞', 'pos1': '自立'},
 {'surface': 'ぬ', 'base': 'ぬ', 'pos': '助動詞', 'pos1': '*'},
 {'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}]

In [86]:
# 形態素解析結果読み込み
def load_mecab(path: str) -> List[List[Dict]]:
    # 読み込み
    text = get_raw(input_path)
    # 文ごとに分離
    sentences = get_sentences(text)
    # マッピング型のリストに格納
    return list(map(mapping_sentence, sentences))

load_mecab(input_path)[0:2]

[[{'surface': '一', 'base': '一', 'pos': '名詞', 'pos1': '数'}],
 [{'surface': '吾輩', 'base': '吾輩', 'pos': '名詞', 'pos1': '代名詞'},
  {'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'},
  {'surface': '猫', 'base': '猫', 'pos': '名詞', 'pos1': '一般'},
  {'surface': 'で', 'base': 'だ', 'pos': '助動詞', 'pos1': '*'},
  {'surface': 'ある', 'base': 'ある', 'pos': '助動詞', 'pos1': '*'},
  {'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}]]

## 31. 動詞
動詞の表層形をすべて抽出せよ．

In [89]:
def get_surface_of_verb(morphemes: List[Dict]) -> List[str]:
    return [m['surface'] for m in morphemes if m['pos'] == '動詞']

test_list = [
    {'surface': 'a', 'pos': '動詞'},
    {'surface': 'b', 'pos': '動詞'},
    {'surface': 'c', 'pos': '名刺'},
]
get_surface_of_verb(test_list) == ['a', 'b']

True

In [ ]:
# load_mecab(input_path)